In [ ]:
import torch

#Learning simple self-attention

inputs = torch.tensor([
    [0.43, 0.15, 0.89],
    [0.55, 0.87, 0.66],
    [0.57, 0.85, 0.64],
    [0.22, 0.58, 0.33],
    [0.77, 0.25, 0.10],
    [0.05, 0.80, 0.55]
])

#Attention weights for the second input - [0.55, 0.87, 0.66], the q,k,v here is the input embedding itself
#Attention weights are - Q.K
query_input = inputs[1]
attention_weight_2 = torch.zeros(len(inputs))

for i,x_i in enumerate(inputs):
    attention_weight_2[i] = torch.dot(query_input, x_i)

print(f"The attention weight for input 2 is: {attention_weight_2}")

Initial atention weight 2 is: tensor([0., 0., 0., 0., 0., 0.])
The attention weight for input 2 is: tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
#Moving on to attention scores = normalized attention weights whose sum is equal to 1
attention_scores_2 = torch.softmax(attention_weight_2, dim=0)
print(f"The attention scores are: {attention_scores_2}")

The attention scores are: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [26]:
#Let's move on to the context vector for input 2.
#Context vector is computed as a combination of all input vectors weighted with respect to input 2
#attention scores = softmax(Q.K), context vector = attention scores . V
attention_scores_2.unsqueeze(0)
context_vector_2 = attention_scores_2 @ inputs
print(f"Shape of the context vector is: {context_vector_2.shape}")

#This can also be done in the following way 
context_vector_2_sim = torch.zeros(query_input.shape)
attention_scores_2.squeeze()

for i,x_i in enumerate(inputs):
    context_vector_2_sim += attention_scores_2[i]*x_i

print(f"Shape of the contenxt_vector without matmul is: {context_vector_2_sim.shape}")
    

Shape of the context vector is: torch.Size([3])
Shape of the contenxt_vector without matmul is: torch.Size([3])
